The concept in this is true ZS. But looks like I accidentally deleted some code blocks. Now unusable. Also smoothness loss not used here.

### True ZS


In [1]:
import numpy as np
import torch
import xarray as xr

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ===== 1. Load Dataset =====
ds = xr.open_dataset('../dat/RBC_Output.nc')  # Adjust path as needed

z_full = ds['z_aac'].values
x_full = ds['x_caa'].values
 
# Select 50 z points (top depths) and 128 evenly spaced x points
z_sel_idx = np.linspace(0, 99, 50, dtype=int)     # Top 100 → select 50
x_sel_idx = np.linspace(0, len(x_full)-1, 128, dtype=int)

z_vals = z_full[z_sel_idx]
x_vals = x_full[x_sel_idx]

def slice_field(field, z_dim, x_dim):
    return np.array(field.isel({z_dim: z_sel_idx, x_dim: x_sel_idx}).values, dtype=np.float32)

max_time_steps = 50  # Use first 50 time steps
b = slice_field(ds['b'].isel(time=slice(0, max_time_steps)), 'z_aac', 'x_caa')
u = slice_field(ds['u'].isel(time=slice(0, max_time_steps)), 'z_aac', 'x_faa')
w = slice_field(ds['w'].isel(time=slice(0, max_time_steps)), 'z_aaf', 'x_caa')
p_dyn = slice_field(ds['p_dyn'].isel(time=slice(0, max_time_steps)), 'z_aac', 'x_caa')

time_vals = np.array(ds['time'].isel(time=slice(0, max_time_steps)).values, dtype=np.float32)

# ===== 2. Compute Δb =====
delta_b = b[1:] - b[:-1]        # (T-1, 50, 128)
time_d = time_vals[1:]          # (T-1,)

# ===== 3. Stack Inputs =====
inputs = np.stack([u[:-1], w[:-1], p_dyn[:-1]], axis=1).astype(np.float32)  # (T-1, 3, 50, 128)
# targets = delta_b.astype(np.float32)                                        # (T-1, 50, 128)
# ===== 2. Use b directly =====
targets = b[:-1].astype(np.float32)  # (T-1, 50, 128) → b instead of Δb


# ===== 4. Quick Normalization =====
subset = min(10, inputs.shape[0])
inputs_mean = inputs[:subset].mean()
inputs_std = inputs[:subset].std()
inputs_norm = (inputs - inputs_mean) / (inputs_std + 1e-8)

targets_mean = targets[:subset].mean()
targets_std = targets[:subset].std()
targets_norm = (targets - targets_mean) / (targets_std + 1e-8)

time_min, time_max = time_d.min(), time_d.max()
time_norm = (time_d - time_min) / (time_max - time_min)

# ===== 5. Convert to Tensors =====
X_tensor = torch.tensor(inputs_norm, dtype=torch.float32).to(device)       # (T-1, 3, 50, 128)
Y_tensor = torch.tensor(targets_norm, dtype=torch.float32).to(device)      # (T-1, 50, 128)
t_tensor = torch.tensor(time_norm, dtype=torch.float32).unsqueeze(1).to(device)  # (T-1, 1)

print(f"✅ Data Loaded → X: {X_tensor.shape}, Y: {Y_tensor.shape}, t: {t_tensor.shape}")


C:\Users\deban\AppData\Local\Temp\ipykernel_32128\72355899.py:8: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset('../dat/RBC_Output.nc')  # Adjust path as needed


✅ Data Loaded → X: torch.Size([49, 3, 50, 128]), Y: torch.Size([49, 50, 128]), t: torch.Size([49, 1])


In [2]:
import numpy as np
import torch

# Set random seed for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

max_time_steps = 100  # Use first 50 time steps

# Select first 50 time steps
X_subset = X_tensor[:max_time_steps]     # (50, 3, 50, 128)
Y_subset = Y_tensor[:max_time_steps]     # (50, 50, 128)
t_subset = t_tensor[:max_time_steps]     # (50, 1)

# Randomly choose 2 distinct time indices (init < target)
time_indices = np.sort(np.random.choice(np.arange(50), size=2, replace=False))
# init_idx, target_idx = time_indices[0], time_indices[0] + 1
init_idx, target_idx = 70, 71

print(f"Init idx: {init_idx}, Target idx: {target_idx}")


Init idx: 70, Target idx: 71


In [3]:
# plot the initial and target b_avg (averaged along x) vs z profiles
import matplotlib.pyplot as plt

def plot_b_profiles(init_idx, target_idx):
    plt.figure(figsize=(10, 6))
    # Ensure tensors are on CPU before converting to numpy
    y_init = Y_subset[init_idx]
    y_target = Y_subset[target_idx]
    if hasattr(y_init, 'cpu'):
        y_init = y_init.cpu().numpy()
    if hasattr(y_target, 'cpu'):
        y_target = y_target.cpu().numpy()
    plt.plot(z_vals, y_init.mean(axis=1), label=f'Init (t={init_idx})', marker='o')
    plt.plot(z_vals, y_target.mean(axis=1), label=f'Target (t={target_idx})', marker='x')
    plt.xlabel('Depth (z)')
    plt.ylabel('b_avg')
    plt.title('b_avg vs Depth Profiles')
    plt.legend()
    plt.grid()
    plt.show()

plot_b_profiles(init_idx, target_idx)


IndexError: index 70 is out of bounds for dimension 0 with size 49

<Figure size 1000x600 with 0 Axes>

In [ ]:
X_init = X_subset[init_idx].unsqueeze(0).to(device)      # (1, 3, 50, 128)
Y_init = Y_subset[init_idx].unsqueeze(0).to(device)      # (1, 50, 128)
t_init = t_subset[init_idx].unsqueeze(0).to(device)      # (1, 1)

X_target = X_subset[target_idx].unsqueeze(0).to(device)  # (1, 3, 50, 128)
Y_target = Y_subset[target_idx].unsqueeze(0).to(device)  # (1, 50, 128)
t_target = t_subset[target_idx].unsqueeze(0).to(device)  # (1, 1)


In [ ]:
import torch
import torch.nn as nn


class DeepONetCNN(nn.Module):

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
    def __init__(self):
        super(DeepONetCNN, self).__init__()

        self.init_weights()
        
        # ===== Branch: Deeper CNN over (3, 50, 128) =====
        self.branch = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),   # (B, 32, 50, 128)
            nn.BatchNorm2d(32),
            nn.ReLU(),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # (B, 64, 50, 128)
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # (B, 128, 50, 128)
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 64, kernel_size=3, padding=1),  # (B, 64, 50, 128)
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64, 1, kernel_size=3, padding=1),    # (B, 1, 50, 128)
            nn.Flatten(start_dim=1),                       # (B, 50*128)
        )
        
        # ===== Trunk: Time embedding to match spatial size =====
        self.trunk = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 50 * 128),  # must match flattened spatial size
        )
        
    def forward(self, x, t):
        B_out = self.branch(x)  # (B, 50*128)
        T_out = self.trunk(t)    # (B, 50*128)
        out = B_out * T_out      # (B, 50*128)
        out = out.view(-1, 50, 128)  # (B, 50, 128)
        return out


In [ ]:
def compute_physics_loss(model, X, t, κ=1e-6, ν=1e-6, dz=0.0078, dx=0.0078):
    pred_b = model(X, t).squeeze(0)  # (50,128)
    u = X[0, 0]
    w = X[0, 1]
    p = X[0, 2]

    # Pad with dummy dim
    def pad2d(f): return torch.nn.functional.pad(f.unsqueeze(0), (1,1,1,1), mode='replicate').squeeze(0)

    b_pad = pad2d(pred_b)
    u_pad = pad2d(u)
    w_pad = pad2d(w)
    p_pad = pad2d(p)

    # Derivatives (central diff)
    b_x = (b_pad[1:-1, 2:] - b_pad[1:-1, :-2]) / (2 * dx)
    b_z = (b_pad[2:, 1:-1] - b_pad[:-2, 1:-1]) / (2 * dz)
    b_xx = (b_pad[1:-1, 2:] - 2 * pred_b + b_pad[1:-1, :-2]) / dx**2
    b_zz = (b_pad[2:, 1:-1] - 2 * pred_b + b_pad[:-2, 1:-1]) / dz**2

    u_x = (u_pad[1:-1, 2:] - u_pad[1:-1, :-2]) / (2 * dx)
    u_zz = (u_pad[2:, 1:-1] - 2 * u + u_pad[:-2, 1:-1]) / dz**2
    u_xx = (u_pad[1:-1, 2:] - 2 * u + u_pad[1:-1, :-2]) / dx**2

    w_z = (w_pad[2:, 1:-1] - w_pad[:-2, 1:-1]) / (2 * dz)
    w_xx = (w_pad[1:-1, 2:] - 2 * w + w_pad[1:-1, :-2]) / dx**2
    w_zz = (w_pad[2:, 1:-1] - 2 * w + w_pad[:-2, 1:-1]) / dz**2

    p_x = (p_pad[1:-1, 2:] - p_pad[1:-1, :-2]) / (2 * dx)
    p_z = (p_pad[2:, 1:-1] - p_pad[:-2, 1:-1]) / (2 * dz)

    # Residuals
    advection_b = u * b_x + w * b_z
    diffusion_b = κ * (b_xx + b_zz)
    residual_b = advection_b - diffusion_b  # ∂b/∂t is zero in steady

    adv_u = u * u_x + w * ((u_pad[2:, 1:-1] - u_pad[:-2, 1:-1]) / (2 * dz))
    diff_u = ν * (u_xx + u_zz)
    mom_x = adv_u + p_x - diff_u

    adv_w = u * ((w_pad[1:-1, 2:] - w_pad[1:-1, :-2]) / (2 * dx)) + w * w_z
    diff_w = ν * (w_xx + w_zz)
    mom_z = adv_w + p_z - diff_w - pred_b

    div = u_x + w_z

    bc_top = torch.mean((pred_b[-1, :] - 0.5) ** 2)
    bc_bottom = torch.mean((pred_b[0, :] - 0.0) ** 2)
    bc_periodic = torch.mean((pred_b[:, 0] - pred_b[:, -1]) ** 2)

    loss_b = torch.mean(residual_b ** 2)
    loss_mom = torch.mean(mom_x ** 2) + torch.mean(mom_z ** 2)
    loss_div = torch.mean(div ** 2)
    loss_bc = bc_top + bc_bottom + bc_periodic

    total_phy = loss_b + 0.5 * loss_mom + 0.1 * loss_div + 0.5 * loss_bc
    return total_phy


In [ ]:
# ==========================
# Model Setup
# ==========================
model = DeepONetCNN().to(device)
model.init_weights()  # Assuming you added the Xavier init method
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
mse = torch.nn.MSELoss()

# ==========================
# Training Loop
# ==========================
epochs = 3000

for epoch in range(epochs):
    model.train()

    pred_init = model(X_init, t_init)  # (1, 50, 128)
    ic_loss = mse(pred_init, Y_init)

    phy_loss = compute_physics_loss(model, X_init, t_init)

    total_loss = ic_loss + phy_loss

    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if epoch % 500 == 0 or epoch == epochs - 1:
        print(f"[Epoch {epoch}] Total: {total_loss.item():.6f} | IC: {ic_loss.item():.6f} | Physics: {phy_loss.item():.6f}")


[Epoch 0] Total: 1.818511 | IC: 0.997685 | Physics: 0.820826
[Epoch 500] Total: 1.569584 | IC: 0.859961 | Physics: 0.709624
[Epoch 1000] Total: 1.497848 | IC: 0.885905 | Physics: 0.611943
[Epoch 1500] Total: 1.488578 | IC: 0.927708 | Physics: 0.560870
[Epoch 2000] Total: 1.573566 | IC: 0.903664 | Physics: 0.669902
[Epoch 2500] Total: 1.495760 | IC: 0.934516 | Physics: 0.561244
[Epoch 2999] Total: 1.476186 | IC: 0.932436 | Physics: 0.543750


In [ ]:
model.eval()
with torch.no_grad():
    pred_target = model(X_init, t_target)
    pred_target_np = pred_target.cpu().numpy().squeeze(0) * targets_std + targets_mean
    true_target_np = Y_target.cpu().numpy().squeeze(0) * targets_std + targets_mean


In [ ]:
import matplotlib.pyplot as plt

b_pred_avg = pred_target_np.mean(axis=1)    # (50,)
b_true_avg = true_target_np.mean(axis=1)    # (50,)

time = time_vals[:50]  # Use first 50 time steps for plotting

plt.figure(figsize=(6,5))
plt.plot(b_true_avg, z_vals, label=f'True b @ t={time[target_idx]:.2e}', marker='o')
plt.plot(b_pred_avg, z_vals, label=f'Pred b @ t={time[target_idx]:.2e}', linestyle='--', marker='x')
plt.gca().invert_yaxis()
plt.xlabel('Δb')
plt.ylabel('Depth (z)')
plt.title('Zero-Shot PINN Prediction')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'pred_target_np' is not defined